# Calculate LPIPS for 10k image pairs from 1 folder 
### Pair: image N & image (N + 1)

In [1]:
# !pip install lpips

In [ ]:
import os
import lpips
import numpy as np
from tqdm import tqdm
import random

# select IRCAD images 
# Since we calculate average distance between 10k pairs of current and consecutive images, it makes sense to have one list specifying which IRCAD frames in which order appear in calculation.
# For that purpose in our paper we choose images generated from model vid25_66 part 1
list_ = os.listdir('/path/to/output_vid25_66_part1')

# Now we can calculate LPIPS for any data. Below example for mixed style, paths can be for inputs or baseline output data.
data_path_25_66 = '/path/to/data' #output_vid25_66
data_path_01_49 = '/path/to/data' #output_vid01_49
data_path_52_56 = '/path/to/data' #output_vid52_56

# calculate FID & KID for mixed style
rand_dict = {1: data_path_25_66,
            2: data_path_01_49,
            3: data_path_52_56}

files_common =set(os.listdir(rand_dict[1]))& set(os.listdir(rand_dict[2])) & set(os.listdir(rand_dict[3]))
files = [os.path.join(rand_dict[random.randint(1, 3)], f) for f in files_common]

random.seed(420420)
files = random.sample(files,10000)

In [ ]:
# Initializing the model
loss_fn = lpips.LPIPS(net='vgg',version='0.1')
loss_fn.to('cuda:1')

dists = []
for (ff,file) in tqdm(enumerate(files[:-1])):

	img0 = lpips.im2tensor(lpips.load_image(file)) # RGB image from [-1,1]
	img0 = img0.to('cuda:1')

	files1 = [files[ff+1],]
	
	for file1 in files1:
		img1 = lpips.im2tensor(lpips.load_image(file1))
		img1 = img1.to('cuda:1')

		# Compute distance
		dist01 = loss_fn.forward(img0,img1)
		dists.append(dist01.item())

avg_dist = np.mean(np.array(dists))
stderr_dist = np.std(np.array(dists))/np.sqrt(len(dists))

print('Avg: %.5f +/- %.5f'%(avg_dist,stderr_dist))

